# Export DevInfo Access Database Shapefiles to GeoJSON

DevInfo stores the three files (.shp, .shx, and .dbf) in 3 columns in the Access database. This script will execute a query that will return those files, plus an additional column, `Parent_NId`, that will be used to construct the file name for the output GeoJSON file. 

## Import the needed python libraries
Install the GDAL pythyon library (ogr import below) by opening the anaconda prompt and using `conda install gdal`

In [ ]:
import os
import shutil
import json
from osgeo import ogr
import pyodbc
from IPython.display import clear_output

### Test to see if you have the needed drivers installed to connect to an Access Database
Run the line below and if you see an emtpy array: `[]` as a result, you may need to install the 64-bit ACE drivers

*for reference https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-Microsoft-Access*

*for 64-bit drivers : https://www.microsoft.com/en-us/download/confirmation.aspx?id=13255*

In [ ]:
[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

## Set defaults

In [ ]:
shp_driver_lbl = 'Esri Shapefile'
shp_driver = ogr.GetDriverByName(shp_driver_lbl)
if shp_driver is None:
    print ('{} driver not available.'.format(shp_driver_lbl))
else:
    print ('{} driver IS available.'.format(shp_driver_lbl))

### Set your working directory
example: *C:\users\me\working_directory\my_country*

In [ ]:
output_working_directory = r'C:\Users\adam6475\devinfo\tanzania'

### Enter the path to your DevInfo Access Database
example: *C:\users\me\working_directory\my_country\my_devinfo_database.mdb (.accdb)*

In [ ]:
access_database = r'C:\Users\adam6475\devinfo\tanzania\TSED_20180423.mdb'

### Enter the name of your output folder for the geojson files
example: *C:\users\me\working_directory\my_country\geojson*

In [ ]:
output_gj_folder = r'C:\Users\adam6475\devinfo\tanzania\geojson'

## Execute the query against the DevInfo Access tables

In [ ]:
connStr = (
    r'Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};'
    r'DBQ={};'.format(access_database)
)

cnxn = pyodbc.connect(connStr)

sql = """\
SELECT 
UT_Area_Map_Layer.Layer_Shp AS SHP, 
UT_Area_Map_Layer.Layer_Shx AS SHX,
UT_Area_Map_Layer.Layer_dbf AS DBF, 
UT_Area_Map_Layer.Layer_NId AS LAYER_NID, 
UT_Area_en.Area_NId AS AREA_NID, 
UT_Area_en.Area_ID AS AREA_ID, 
UT_Area_en.Area_Parent_NId AS PARENT_NID, 
UT_Area_en.Area_Name AS AREA_NAME, 
UT_Area_Level_en.Area_Level AS AREA_LEVEL, 
UT_Area_Level_en.Area_Level_Name AS AREA_LEVEL_NAME

FROM 
UT_Area_Level_en INNER JOIN (UT_Area_Map_Layer INNER JOIN (UT_Area_en INNER JOIN UT_Area_Map ON UT_Area_en.Area_NId = UT_Area_Map.Area_NId) ON UT_Area_Map_Layer.Layer_NId = UT_Area_Map.Layer_NId) ON UT_Area_Level_en.Area_Level = UT_Area_en.Area_Level

ORDER BY UT_Area_en.Area_Parent_NId  ASC

"""
crsr = cnxn.execute(sql)

# export just a subset for testing
# rows = crsr.fetchmany(5)

# uncomment this to get crazy and export them all
rows = crsr.fetchall()

print ('sucessfully executed data query :: {} rows returned'.format(len(rows)))

## Group the result of the query by parent id

In [ ]:
chunks = {}
for row in rows:
    
    parent_id = row.PARENT_NID
    
    if parent_id not in chunks:
        chunks[parent_id] = {}
        chunks[parent_id]['rows'] = []
            
    chunks[parent_id]['rows'].append(row)

print ('done chunking data by parent')

# uncomment to view the number of rows for each parent
# for c in chunks:
#     print (c, len(chunks[c]['rows']))

## For each parent group, create a GeoJSON file with its features

In [ ]:
temp_folder_shp = os.path.abspath('{}/temp'.format(output_gj_folder))

try:
    os.mkdir(temp_folder_shp)
except:
    print ('unable to make temp shapes dir. may already exist')

debug = []

print ('exporting to geojson ...')

for parent in chunks:
    # test with just one parent
#     if str(parent) != '86':
#         continue
        
    msg = 'PARENT {}'.format(parent)
#     print (msg)
    debug.append(msg)
    
    feature_collection = {
        'type' : 'FeatureCollection',
        'features': []
    }
    
    rows = chunks[parent]['rows']
        
    for i, row in enumerate(rows):
        shp = row[0]
        shx = row[1]
        dbf = row[2]
        
        layer_id = row[3]
        parent_id = row[6]
        area_id = row[5]
        area_name = row[7]
        area_level = row[8]
        
        # get geometry
        out_shp_path = os.path.abspath('{}/{}'.format(temp_folder_shp, area_id))
        out_shp_path_ext = '{}.shp'.format(out_shp_path)
            
        with open('{}.shp'.format(out_shp_path), 'wb') as writer:
            writer.write(shp)
        with open('{}.shx'.format(out_shp_path), 'wb') as writer:
            writer.write(shx)
        with open('{}.dbf'.format(out_shp_path), 'wb') as writer:
            writer.write(dbf)
        # here we are also writing a GCS WGS 1984 .prj file that will define the spatial reference of each shapefile
        with open('{}.prj'.format(out_shp_path), 'w') as writer:
            writer.write('GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]')
        
        # open the newly created area shapefile
        area_shp_file = shp_driver.Open(out_shp_path_ext, 0)
        area_layer = area_shp_file.GetLayer()
        
        feature_count = area_layer.GetFeatureCount()
        
        feature = None
        if feature_count > 1:
            where_clause = 'ID_ = \'{}\''.format(area_id)
            area_layer.SetAttributeFilter(where_clause)
            found_feature = area_layer.GetNextFeature()
            if found_feature is None:
                msg = 'unable to get feature with area_id :: {}'.format(area_id)
#                 print (msg)
                debug.append(msg)
                continue
            else:
                feature = found_feature
                            
        else:
            feature = area_layer.GetFeature(0)
        
        geom_ref = feature.GetGeometryRef()
        geom_json_str = geom_ref.ExportToJson()
        geom = json.loads(geom_json_str)
        
        # save and close the shapefile
        area_shp_file = None

        # setup the new feature
        feature = {
            'type': 'Feature',
            'properties': {
                'REF_AREA_ID': area_id,
                'REF_AREA': area_name,
                'LEVEL': area_level,
                'PARENT_NID': parent,
                'LAYER_NID': layer_id
            },
            'geometry': geom
        }

        feature_collection['features'].append(feature)
    
    out_parent_gj_filename = 'parent_{}.geojson'.format(str(parent))
    out_parent_gj_path = os.path.abspath(os.path.join(output_gj_folder, out_parent_gj_filename))
    
    msg = 'writing {} features to {}'.format(len(feature_collection['features']), out_parent_gj_path)
#     print (msg)
    debug.append(msg)
    
    with open(out_parent_gj_path, 'w') as parent_gj_file:
        parent_gj_file.write(json.dumps(feature_collection))     

# delete the temp shapefile folders
shutil.rmtree(temp_folder_shp, ignore_errors=True)

print ('done')       

---

## view log results (optional)
if you have the pandas library installed, [more info here](http://pandas.pydata.org/pandas-docs/stable/install.html), you can view the results in a table here

In [ ]:
import pandas as pd
pd.set_option('max_colwidth', 1000)

df = pd.DataFrame(debug, columns=['message'])

view in notebook

In [ ]:
df.style.hide_index()

export to csv

In [ ]:
file_name = 'testing/log.csv'
df.to_csv(file_name)